In [1]:
import pandas as pd
import numpy as np

In [11]:
movies = pd.read_csv('movies.csv', usecols=['movieId', 'title'] ,dtype={'movieId': 'int32', 'title': 'str'} )

In [4]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
ratings = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'] , dtype = {'userId' : 'int32' , 'movieId' : 'int32' , 'rating':'float32' })

In [6]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float32
dtypes: float32(1), int32(2)
memory usage: 1.2 MB


In [13]:
df = pd.merge(movies ,ratings , on = 'movieId')

In [14]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [15]:
combined_movie_rating = df.dropna(axis = 0 , subset = ['title'] )

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int32  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int32  
 3   rating   100836 non-null  float32
dtypes: float32(1), int32(2), object(1)
memory usage: 2.7+ MB


In [16]:
combined_movie_rating.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [18]:
combined_movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int32  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int32  
 3   rating   100836 non-null  float32
dtypes: float32(1), int32(2), object(1)
memory usage: 2.7+ MB


In [26]:
movieRatingCount = (combined_movie_rating.groupby(['title'])['rating'].count().reset_index().rename(columns = {'rating':'totalMovieRating'})[['title', 'totalMovieRating']])

In [27]:
movieRatingCount.head()

,title,totalMovieRating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [29]:
rating_with_totalRatingCount = combined_movie_rating.merge(movieRatingCount , on = 'title' , how = 'left')

In [30]:
rating_with_totalRatingCount.head()

,movieId,title,userId,rating,totalMovieRating
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [40]:
popularity_threshold = 50

In [42]:
rating_ofpopularmovie = rating_with_totalRatingCount.query('totalMovieRating > @popularity_threshold')

In [44]:
data = rating_ofpopularmovie.pivot_table(index = 'title' , columns = 'userId'  , values = 'totalMovieRating').fillna(0)

In [46]:
from sklearn.neighbors import  NearestNeighbors

In [47]:
mode_knn = NearestNeighbors(metric= 'cosine' )

In [48]:
mode_knn.fit(data)

NearestNeighbors(metric='cosine')

In [51]:
data.iloc[350,:].values.reshape(-1,1)

array([[279.],
       [  0.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [279.],
       [279.],
       [  0.],
       [  0.],
       [279.],
       [  0.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [279.],
       [279.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [  0.],
       [  0.],
       [279.],
       [279.],
       [  0.],
       [  0.],
       [  0.],
       [279.],
       [279.],
       [279.],
       [  0.],
       [279.],
       [279.],
       [  0.],
       [  0.],
       [279.],
       [279.],
       [279.],
       [  0.],
       [  0.],
       [279.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [279.],
       [  0.],
       [27

In [59]:
distance , indices  = mode_knn.kneighbors(data.iloc[350,:].values.reshape(1,-1),  n_neighbors  = 5 )

In [71]:
j = 0
for i in indices :

 print(data.index[i] ,"Distance",distance[j]   )
 j=j+1

Index(['Silence of the Lambs, The (1991)', 'Pulp Fiction (1994)',
       'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)',
       'Seven (a.k.a. Se7en) (1995)'],
      dtype='object', name='title') Distance [3.33066907e-16 2.92707495e-01 3.30853295e-01 3.43169931e-01
 3.69708953e-01]
